# HW5

In [219]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ColorConverter

import pandas as pd
import nltk
import string
from collections import OrderedDict


####We might not need all of these####
import sklearn
from sklearn import datasets, feature_selection, cluster, feature_extraction
from sklearn import neighbors, decomposition, metrics, preprocessing
from sklearn import cross_validation, grid_search 
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import MDS
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from IPython import display
from PIL import Image
import os
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
#######################################


# Using AFINN library - pip install afinn
from afinn import Afinn

# Load the NLTK punkt tokenizer - uncomment the next line if you don't have the punkt tokenizer 
# installed - it will open a dialog that will allow you to do so.
#nltk.download()


In [220]:
# Read in Shakespeare's plays
df = pd.read_pickle('data.pickle')

In [221]:
#print(afinn_scores)
print(df.shape)
print(df['Utterance'][3])
#df['Utterance'] = df['Utterance'] + ""
print(df['Utterance'][3])

(30919, 5)
i'll set a bourn how far to be beloved. 
i'll set a bourn how far to be beloved. 


In [222]:
df.describe

<bound method NDFrame.describe of            Play  Act  Scene             Speaker  \
0       a_and_c  0.0    0.0               PHILO   
1       a_and_c  0.0    0.0           CLEOPATRA   
2       a_and_c  0.0    0.0         MARK ANTONY   
3       a_and_c  0.0    0.0           CLEOPATRA   
4       a_and_c  0.0    0.0         MARK ANTONY   
5       a_and_c  0.0    0.0           Attendant   
6       a_and_c  0.0    0.0         MARK ANTONY   
7       a_and_c  0.0    0.0           CLEOPATRA   
8       a_and_c  0.0    0.0         MARK ANTONY   
9       a_and_c  0.0    0.0           CLEOPATRA   
10      a_and_c  0.0    0.0         MARK ANTONY   
11      a_and_c  0.0    0.0           CLEOPATRA   
12      a_and_c  0.0    0.0         MARK ANTONY   
13      a_and_c  0.0    0.0           CLEOPATRA   
14      a_and_c  0.0    0.0         MARK ANTONY   
15      a_and_c  0.0    0.0           DEMETRIUS   
16      a_and_c  0.0    0.0               PHILO   
17      a_and_c  0.0    0.0           DEMETRIUS 

In [223]:
dfAllPlays = df[['Speaker','Utterance']]
dfAllPlays = dfAllPlays.groupby(['Speaker']).agg({'Utterance':'sum'})
dfAllPlays.reset_index(inplace=True)
dfAllPlays 
dfAllPlays['Speaker'][414]
dfAllPlays['Utterance'][414]
#dfAllPlays.shape

"  a little more than kin, and less than kind. not so, my lord; i am too much i' the sun. ay, madam, it is common. seems, madam! nay it is; i know not 'seems.' 'tis not alone my inky cloak, good mother, nor customary suits of solemn black, nor windy suspiration of forced breath, no, nor the fruitful river in the eye, nor the dejected 'havior of the visage, together with all forms, moods, shapes of grief, that can denote me truly: these indeed seem, for they are actions that a man might play: but i have that within which passeth show; these but the trappings and the suits of woe. i shall in all my best obey you, madam. o, that this too too solid flesh would melt thaw and resolve itself into a dew! or that the everlasting had not fix'd his canon 'gainst self-slaughter! o god! god! how weary, stale, flat and unprofitable, seem to me all the uses of this world! fie on't! ah fie! 'tis an unweeded garden, that grows to seed; things rank and gross in nature possess it merely. that it should c

In [224]:
dfAllPlays['Utterance'] = dfAllPlays.Utterance.str.replace("[^a-zA-Z]", " ",)  #Remove all punctuations/numbers. 

dfAllPlays['Utterance'][414]

'  a little more than kin  and less than kind  not so  my lord  i am too much i  the sun  ay  madam  it is common  seems  madam  nay it is  i know not  seems    tis not alone my inky cloak  good mother  nor customary suits of solemn black  nor windy suspiration of forced breath  no  nor the fruitful river in the eye  nor the dejected  havior of the visage  together with all forms  moods  shapes of grief  that can denote me truly  these indeed seem  for they are actions that a man might play  but i have that within which passeth show  these but the trappings and the suits of woe  i shall in all my best obey you  madam  o  that this too too solid flesh would melt thaw and resolve itself into a dew  or that the everlasting had not fix d his canon  gainst self slaughter  o god  god  how weary  stale  flat and unprofitable  seem to me all the uses of this world  fie on t  ah fie   tis an unweeded garden  that grows to seed  things rank and gross in nature possess it merely  that it should c

In [225]:
dfAllPlays

,Speaker,Utterance
0,A Lord,sir it was i
1,A Patrician,this man has marr d his fortune you do the no...
2,AARON,now climbeth tamora olympus top safe out of ...
3,ABERGAVENNY,i cannot tell what heaven hath given him let...
4,ABHORSON,do you call sir a bawd sir fie upon him h...
5,ABRAHAM,do you bite your thumb at us sir do you bite...
6,ACHILLES,why how now ajax wherefore do you thus how...
7,ADAM,yonder comes my master your brother sweet ma...
8,ADRIAN,though this island seem to be desert uninha...
9,ADRIANA,neither my husband nor the slave return d tha...


In [226]:
porter = PorterStemmer()
def tokenizer_porter(text):
    #for word in text.split():
    #    print(word, porter.stem(word))
    return [porter.stem(word) for word in text.split()]

all_stopwords = stopwords.words('english') 
#adding Shakespeare-ean stop-words
shaksp_stopwords = ('thee', 'thou', 'thy', 'thine')
all_stopwords += shaksp_stopwords
print(all_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [227]:
#TfidfVectorizer - Convert a collection of raw documents to a matrix of TF-IDF features.
vectorizer = TfidfVectorizer(tokenizer = tokenizer_porter,
                             stop_words=all_stopwords,
                             use_idf=False,
                             max_features = 100,ngram_range=(1,1))
train_data_features = vectorizer.fit_transform(dfAllPlays.Utterance.values)
train_data_features = train_data_features.A #toarray()  
vocab = vectorizer.get_feature_names()

In [228]:
#vocab
train_data_features

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.0124844 ,  0.0124844 ,  0.06242202, ...,  0.02496881,
         0.14981285,  0.06242202],
       ..., 
       [ 0.26726124,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.26413527,  0.        ],
       [ 0.        ,  0.        ,  0.70710678, ...,  0.        ,
         0.        ,  0.        ]])

In [229]:
df = pd.DataFrame()
df = pd.DataFrame(data=train_data_features,columns=vocab)
df

,ani,art,away,ay,bear,befor,blood,brother,call,cannot,...,us,veri,wa,way,well,whi,word,world,would,yet
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.012484,0.012484,0.062422,0.062422,0.049938,0.049938,0.062422,0.062422,0.037453,0.037453,...,0.062422,0.049938,0.087391,0.024969,0.074906,0.149813,0.024969,0.024969,0.149813,0.062422
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.107211,0.000000,0.000000,0.000000,0.000000,0.107211,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107211,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.026919,0.053838,0.026919,0.026919,0.026919,0.000000,0.026919,0.000000,0.053838,0.000000,...,0.053838,0.026919,0.053838,0.000000,0.026919,0.134595,0.026919,0.000000,0.053838,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.051988,0.051988,0.051988,0.155963,0.103975,0.051988,...,0.000000,0.000000,0.051988,0.000000,0.051988,0.207950,0.051988,0.051988,0.103975,0.207950
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.267261,0.000000,0.000000,0.000000,0.000000,...,0.267261,0.000000,0.534522,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.267261
9,0.031928,0.031928,0.079819,0.063855,0.127710,0.000000,0.015964,0.000000,0.031928,0.031928,...,0.111746,0.000000,0.095783,0.000000,0.047891,0.047891,0.015964,0.000000,0.175601,0.047891


In [230]:
CS_dist = 1 - cosine_similarity(train_data_features)
Euclid_dist = euclidean_distances(train_data_features)

num_samples, num_features = train_data_features.shape
print ("num_samples:  %d, num_features: %d" %(num_samples,num_features))

num_samples:  971, num_features: 100


In [231]:
print(df.shape, Euclid_dist.shape, CS_dist.shape)

(971, 100) (971, 971) (971, 971)


In [232]:
df.head()

,ani,art,away,ay,bear,befor,blood,brother,call,cannot,...,us,veri,wa,way,well,whi,word,world,would,yet
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.012484,0.012484,0.062422,0.062422,0.049938,0.049938,0.062422,0.062422,0.037453,0.037453,...,0.062422,0.049938,0.087391,0.024969,0.074906,0.149813,0.024969,0.024969,0.149813,0.062422
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.107211,0.000000,0.000000,0.000000,0.000000,0.107211,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107211,0.000000


### Computing MDS coefficients
Using MDS for the dimension reduction of the features. This has the advantage of using the cosine similarity of the TF-IDF matrix (transformed to dissimilarity) for the calculation of distances.


In [233]:
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=99)
text_MDS = mds.fit_transform(CS_dist)  # shape (n_components, n_samples)
text_MDS.shape

(971, 2)

In [234]:
text_MDS

array([[ 0.70705206,  0.09815038],
       [-0.55539272, -0.16149528],
       [-0.20602817,  0.06983596],
       ..., 
       [-0.3268803 ,  0.68465437],
       [ 0.26088695, -0.28166633],
       [ 0.44617366,  0.59846771]])

Performing K-Means on the TF-IDF matrix with the K++ initialisation and 2 clusters.

In [235]:
MAX_CLUSTERS = 2
kmeans = KMeans(init='k-means++', n_clusters=MAX_CLUSTERS, n_init=10)
kmeans.fit(train_data_features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [236]:
df

,ani,art,away,ay,bear,befor,blood,brother,call,cannot,...,us,veri,wa,way,well,whi,word,world,would,yet
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.012484,0.012484,0.062422,0.062422,0.049938,0.049938,0.062422,0.062422,0.037453,0.037453,...,0.062422,0.049938,0.087391,0.024969,0.074906,0.149813,0.024969,0.024969,0.149813,0.062422
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.107211,0.000000,0.000000,0.000000,0.000000,0.107211,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.107211,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.026919,0.053838,0.026919,0.026919,0.026919,0.000000,0.026919,0.000000,0.053838,0.000000,...,0.053838,0.026919,0.053838,0.000000,0.026919,0.134595,0.026919,0.000000,0.053838,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.051988,0.051988,0.051988,0.155963,0.103975,0.051988,...,0.000000,0.000000,0.051988,0.000000,0.051988,0.207950,0.051988,0.051988,0.103975,0.207950
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.267261,0.000000,0.000000,0.000000,0.000000,...,0.267261,0.000000,0.534522,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.267261
9,0.031928,0.031928,0.079819,0.063855,0.127710,0.000000,0.015964,0.000000,0.031928,0.031928,...,0.111746,0.000000,0.095783,0.000000,0.047891,0.047891,0.015964,0.000000,0.175601,0.047891


In [237]:
df['KM_cluster'] = kmeans.predict(train_data_features)
#df['KM_cluster']

In [239]:
# Set the cluster names
cluster_names = {0: 'Cluster 1', 1: 'Cluster 2', 2: 'Cluster 3'}
df['KM_cluster'].replace(cluster_names, inplace=True)
#df['KM_cluster']
#list(df.columns.values)

In [240]:
print("\nTotal nummber of clusters and composition: ")
print (df['KM_cluster'].value_counts()[::-1])
print("\nSilhouette Coefficient: %0.3f"
      % metrics.silhouette_score(train_data_features, kmeans.labels_, sample_size=1000))


Total nummber of clusters and composition: 
Cluster 1    416
Cluster 2    555
Name: KM_cluster, dtype: int64

Silhouette Coefficient: 0.123


In [241]:
df['Character'] = dfAllPlays['Speaker']
df[['Character', 'KM_cluster']]

,Character,KM_cluster
0,A Lord,Cluster 1
1,A Patrician,Cluster 2
2,AARON,Cluster 2
3,ABERGAVENNY,Cluster 2
4,ABHORSON,Cluster 1
5,ABRAHAM,Cluster 1
6,ACHILLES,Cluster 2
7,ADAM,Cluster 2
8,ADRIAN,Cluster 1
9,ADRIANA,Cluster 2


In [242]:
afinn = Afinn()

# For every utterance, add a Valence score
afinn_scores = [afinn.score(text) for text in df.Utterance]

df['afinn'] = afinn_scores

AttributeError: 'DataFrame' object has no attribute 'Utterance'

## Differentiating comedies from tragedies

The paper first attempts to differentiate comedies from tragedies based on the summed valence for each play.

In [ ]:
awv = {}

for play in df.Play.unique():
    dfp = df[df['Play']==play]
    
    #Haris
    #print (play, df['Play'])
    #print ("dfp", dfp)

    # Sum the valences for each of the utterances
    sum_valence = dfp.afinn.sum()
    
    # Generate one big string for the full play
    alltext = ''
    for text in dfp['Utterance']:
        alltext += text + ' '
    
    
    # The following lines can be added if you wish to take into account the play's length
    #
    
    # Create a dictionary using a comprehension - this maps every character from
    # string.punctuation to None. Initialize a translation object from it.
    #translator = str.maketrans({key: None for key in string.punctuation})

    # Remove the punctuation using the translator
    #no_punctuation = alltext.translate(translator)
    
    # Split the play into tokens
    #tokens = nltk.word_tokenize(no_punctuation)

    # Calculate the number of words in the play
    #len_play = len(tokens)
    
    
    # Find the number of words in AFINN that are in the play
    words_in_afinn = afinn.find_all(alltext)


    # Calculate the average word valence
    # The paper seems to infer there are two methods to calculate it:
    # i) "Fig. 1. For each play the valence for each word was summed and then divided by the number
    # of words in both the word list (AFINN) and the respective play."
    # ii) "...  by summing the valence values for all words in the play and then dividing by the number 
    # of the play’s words in AFINN"
    #
    # We've chosen the later.
    awv[play] = (sum_valence / len(words_in_afinn)) 

    
# Define the list of tragedies and comedies in the order given in the paper
tragedies = ['titus','r_and_j', 'j_caesar', 'hamlet', 'othello', 'timon', 'lear', 'macbeth', 'a_and_c', 'coriolan']
comedies = ['all_well', 'as_you', 'com_err', 'lll', 'm_for_m', 'merchant', 'm_wives', 'dream', 'much_ado', 'pericles',
           'taming', 't_night', 'two_gent', 'win_tale']
all_labels = comedies + tragedies 

awv_sorted = OrderedDict()
for name in comedies:
    awv_sorted[name] = awv[name]
    
for name in tragedies:
    awv_sorted[name] = awv[name]

In [ ]:
plt.rcParams["figure.figsize"] = (16,9)

bar=plt.bar(range(len(awv_sorted)), awv_sorted.values(), align='center', color='blue')

# Change the color of the tragedies to red
for i in range(len(comedies),len(comedies)+len(tragedies)):
    bar[i].set_color('red')

plt.xticks(range(len(all_labels)), all_labels)

ax = plt.axes()
ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=90)
plt.show()

## Character-to-Character Analysis

The paper next attempts to determine the sentiment between characters. The character-to-character sentiment is determined by "summing the valence values over each instance of continuous speech and then assumed that sentiment was directed towards the character that spoke immediately before the current speaker." Our data structure is ordered by speaker and so we can simply iterate through it and calculate the valence.


In [ ]:
from_character = {}
speaker = ''
previous_speaker = ''

# We'll look at character interactions in Hamlet
dfp = df[df.Play=='hamlet']

# Iterate through all Utterances
for i, r in dfp.iterrows():

    # Determine who is speaking
    speaker = r['Speaker']

    # Only calculate if someone spoke before
    if len(previous_speaker) > 0:
        # Get the list of speakers that this speaker has spoken to
        if speaker in from_character:
            to_character = from_character[speaker]
        else:
            to_character = {}
            from_character[speaker] = to_character

        # Get the running total of sentiment valence so far and add the new valence
        if previous_speaker in to_character:
            to_character[previous_speaker] += r['afinn']
        else:
            to_character[previous_speaker] = r['afinn']

    # The current speaker will become the previous speaker for the next utterance
    previous_speaker = speaker

In [ ]:
# Print out Hamlet's Sentiment Valence Sum 
from IPython.display import display, HTML
df_print = pd.DataFrame(list(from_character['HAMLET'].items()), columns=['Speaker', 'Valence'])

df_print.sort_values('Valence',inplace=True, ascending=False)
display(df_print)

In [ ]:
# Print out Hamlet's Sentiment Valence Sum 

df_print2 = pd.DataFrame(list(from_character['GUILDENSTERN'].items()), columns=['Speaker', 'Valence'])

df_print2.sort_values('Valence',inplace=True, ascending=False)
display(df_print2)

In [ ]:
import networkx as nx

G=nx.Graph()

for i, r in df_print.iterrows():
    G.add_edge('HAMLET',r['Speaker'],weight=r['Valence'])


elarge=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] >0]
esmall=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] <=0]

pos=nx.spring_layout(G) # positions for all nodes

# nodes
nx.draw_networkx_nodes(G,pos,node_size=500)

# edges
nx.draw_networkx_edges(G,pos,edgelist=elarge,
                    width=6)
nx.draw_networkx_edges(G,pos,edgelist=esmall,
                    width=6,alpha=0.5,edge_color='b',style='dashed')

# labels
nx.draw_networkx_labels(G,pos,font_size=10,font_family='sans-serif')

plt.axis('off')
plt.show() # display